# Plot all slopes scenarios
Stupid notebook to plot all slopes in one pretty chart

In [1]:
%matplotlib widget

In [2]:
import numpy as np
import math
import bezier
import cmocean.cm as cmo
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)

In [3]:
def slopeFunction(slope_radians, length):
    return math.tan(slope_radians) * length

In [4]:
# TODO improvement: Define whole channel dimensions in gridsteps!
channel = {
    "width": 400,    # [m]  has to be at least x gridstep wide
    "slope": 1.00,   # [deg]
    "length": 15000, # [m] has to be multiple of y gridstep
    "depth": 150,     # [m] 
    "basinslope": 0.2,         # [m] the 'basin' slope    
}

grid = {
    "length": 36000,      # y [m] has to be multiple of y gridstep
    "width":  26000,       # x [m] has to be multiple of x gridstep
    "x_gridstep": 200,    # [m]
    "y_gridstep": 200,    # [m]
    "dims": [],
}

# in the y direction of the grid that is
ylist = np.array([i for i in range(0, grid['length'] + grid['y_gridstep'], grid['y_gridstep'])]) + 100 # + 100 is default start y in REFGRID

In [5]:
# initial_depth = 150 
basinSlope = 0.2

In [6]:
break_point_index = int(channel['length']/grid['y_gridstep'])
smoothen_over = 10

# reduced version of that in JulesD3D
def makeCrossSection(channelSlope, basinSlope, initial_depth=150):
    channel_slope_radians = math.radians(channelSlope)

    channel_slope_range = range(0, channel['length'] + grid['y_gridstep'], grid['y_gridstep'])    
    
    # channel slope depths
    channel_slope_list = np.array([slopeFunction(channel_slope_radians, i) for i in channel_slope_range])

    channel_slope_last_depth = channel_slope_list[-1] + channel['depth']
    
    basin_slope_radians = math.radians(basinSlope)
    basin_length = grid['length'] - channel['length'] 
    basin_list_range = range(0, basin_length, grid['y_gridstep'])
    basin_list = np.array([slopeFunction(basin_slope_radians, i) for i in basin_list_range]) + channel_slope_last_depth
    
    
    channel_slope_list += channel['depth']
    
    depth_list = np.concatenate((channel_slope_list, basin_list), axis=0) + initial_depth

    normal_cross_section = [ylist, depth_list] # x, y values for smoothening
    
    # ----smoothen slope break -----
    x_cross_section = ylist.copy()
    depth_cross_section = depth_list.copy()

    # Smooth with bezier curve between these points
    start_smooth_index, end_smooth_index = [break_point_index - smoothen_over,\
                                            break_point_index + smoothen_over]

    # Prepare section to be smoothed for Bezier
    nodes_x = np.array(x_cross_section[start_smooth_index:end_smooth_index])
    nodes_y = np.array(depth_cross_section[start_smooth_index:end_smooth_index])

    # Feed nodes into bezier instance
    nodes = np.array([nodes_x, nodes_y]) # Bezier class wants it like this
    curved = bezier.Curve.from_nodes(nodes)

    # Get new depth (y-values in bezier funct) from bezier instance
    s_vals_channel = np.linspace(0.0, 1.0, 2 * smoothen_over)
    smoothened_channel_part = curved.evaluate_multi(s_vals_channel)[1]
    
    fig_q, ax_q = plt.subplots()
    fig_q.suptitle('Bathymetry cross-sections (Unsmoothened!)')
    ax_q.plot(range(start_smooth_index, end_smooth_index), -smoothened_channel_part)
    ax_q.set_xlabel('N (grid number)')
    ax_q.set_ylabel('Depth [m]')
    ax_q.grid()    

    smooth_cross_section = depth_cross_section.copy()
    smooth_cross_section[start_smooth_index:end_smooth_index] = smoothened_channel_part
    
    return smooth_cross_section

In [17]:
depths085 = makeCrossSection(0.85, basinSlope)
depths100 = makeCrossSection(1, basinSlope)
depths125 = makeCrossSection(1.25, basinSlope)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
depths1_25.shape

(181,)

In [9]:
# fig, ax = plt.subplots(figsize=(10, 5))
# fig.suptitle('Slopes of different model scenarios', fontsize=16)
# ax.set_title('Smoothened slope breaks')
# # ax.set_aspect('equal')

# ax.set_xlim((0, 36000))
# ax.set_ylim(700, 300)

# ax.plot(ylist, -depths1_00[:-1])

# arrowprops = dict(facecolor='black', shrink=0.05),

# text_100 = ax.annotate('Slope 1.00°', xy=(10000, 450), xytext=(12000, 400), fontsize=13, arrowprops=dict(facecolor='black', shrink=0.02)) #, arrowprops=arrowprops) # 
# text_100.set_path_effects([PathEffects.withStroke(linewidth=1.5, foreground='bisque')])

# ax.plot(ylist, depths1_125[:-1])
# # text_100 = ax.text(10000, -560, "Slope 1.125°", fontsize=13) #, rotation=-8)
# # text_125.set_path_effects([PathEffects.withStroke(linewidth=1.5, foreground='w')])
# text_1125 = ax.annotate('Slope 1.0°', xy=(10000, 450), xytext=(12000, 400), fontsize=13, arrowprops=dict(facecolor='black', shrink=0.02)) #, arrowprops=arrowprops) # 
# text_1125.set_path_effects([PathEffects.withStroke(linewidth=1.5, foreground='bisque')])

# ax.plot(ylist, depths1_25[:-1])
# text_125 = ax.text(9000, -600, "Slope 1.25°", fontsize=13) #, rotation=-8)
# text_125.set_path_effects([PathEffects.withStroke(linewidth=1.5, foreground='w')])


# ax.set_xlabel('Length $n$ [m]')
# ax.set_ylabel('Depth [m]')
# # ax.grid()

In [10]:
plt.close('all')

In [23]:
fig, ax = plt.subplots(figsize=(10, 5))
fig.suptitle('Slopes of Different Scenarios', fontsize=16)
# ax.set_title('Smoothened slope breaks')
# ax.set_aspect('equal')

ax.set_xlim((0, 36000))
ax.set_ylim(700, 300)
ax.xaxis.set_minor_locator(MultipleLocator(1000))
ax.yaxis.set_minor_locator(MultipleLocator(10))


ax.plot(ylist, depths100, c="orange", marker='+', markersize=2.8)
ax.plot(ylist, depths125, c='olive', marker='+', markersize=2.8)

arrow_props = dict(width=1, headwidth=8, facecolor='black', shrink=0.02)
text_085 = ax.annotate('Slope 0.85·°', xy=(14500, 510), xytext=(15500, 460), fontsize=13,  arrowprops=arrow_props) #, arrowprops=arrowprops) # 
text_085.set_path_effects([PathEffects.withStroke(linewidth=0.5, foreground='blueviolet')])

text_100 = ax.annotate('Slope 1.00°', xy=(10000, 470), xytext=(11000, 430), fontsize=13, arrowprops=arrow_props) #, arrowprops=arrowprops) # 
text_100.set_path_effects([PathEffects.withStroke(linewidth=0.5, foreground='orange')])

ax.plot(ylist, depths085, c="blueviolet",  marker='+', markersize=2.8)
text_125 = ax.annotate('Slope 1.25°', xy=(8000, 475), xytext=(1500, 540), horizontalalignment='left',
                       fontsize=13, arrowprops=arrow_props)
text_125.set_path_effects([PathEffects.withStroke(linewidth=0.5, foreground='olive')])


ax.set_xlabel('Length $n$ [m]', fontsize=14)
ax.set_ylabel('Depth [m]', fontsize=14)
# ax.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Depth [m]')

In [ ]:
# plt.close("all")